In [ ]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import csv

In [ ]:
df = pd.read_csv('c:/Users/Tinanope28/Box/Mini Project 2-3/Licensed_Contractors.csv', low_memory = False)
conn = sqlite3.connect ('normalized.db')
cursor = conn.cursor()


In [ ]:
input_file = 'c:/Users/Tinanope28/Box/Mini Project 2-3/Licensed_Contractors.csv'
output_file = 'Licensed_Contractors.tsv'

with open(input_file, 'r', newline='', encoding='utf-8') as csv_in,\
     open(output_file, 'w', newline='', encoding='utf-8') as tsv_out:
    
    reader = csv.reader(csv_in)
    writer = csv.writer(tsv_out, delimiter='\t', quoting=csv.QUOTE_MINIMAL)

    for row in reader:
        writer.writerow(row)

In [ ]:
data_filename = ('c:/Users/Tinanope28/Box/Mini Project 2-3/Licensed_Contractors.tsv')
normalized_database_filename = 'normalized.db'

In [5]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name = None):
    
  if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
    try:
      c = conn.cursor()
      c.execute("DROP TABLE IF EXISTS {}".format(drop_table_name))
    except Error as e:
      print(e)
  
  try:
    c = conn.cursor()
    c.execute(create_table_sql)
  except Error as e:
    print(e)


def execute_sql_statement(sql_statement, conn):
  cur = conn.cursor()
  cur.execute(sql_statement)

  rows = cur.fetchall()

  return rows


In [ ]:
#Step 1 Create State Table
def create_state_table(data_filename, normalized_database_filename):

  state_list = []

  with open (data_filename) as file: 
    header = None

    for line in file:
      if not line.strip():
        continue
        
      if not header:
        header = line.strip().split('\t')
        continue

      data = line.strip().split('\t')
      state = data[9].upper()

      state_tuple = (state,)

      if state_tuple not in state_list:
        state_list.append(state_tuple)


  create_state_table_sql = '''
    CREATE TABLE [State] (
    [StateID] INTEGER NOT NULL PRIMARY KEY,
    [State] TEXT NOT NULL
    );
    '''
  
  def insert_state(conn, values):
    sql = 'INSERT into State (State) VALUES (?)'
    cur = conn.cursor()
    cur.executemany(sql,values)
    return (cur.lastrowid)
  
  conn = create_connection(normalized_database_filename)
  
  with conn:
    create_table(conn, create_state_table_sql, "State")
    insert_state(conn, state_list)
  conn.close()


create_state_table(data_filename, normalized_database_filename)

df = pd.read_sql_query("SELECT * FROM State", conn)
display(df)

,StateID,State
0,1,LA
1,2,SD
2,3,CA
3,4,NY
4,5,VA
5,6,PA
6,7,GA
7,8,RD
8,9,MN
9,10,NC


In [ ]:
# Create State to State ID and State ID to State Dictionaries
def create_state_to_stateID_dictionary(normalized_database_filename):

    conn = create_connection(normalized_database_filename)

    df = pd.read_sql_query ("SELECT * FROM State", conn)

    state_to_stateID_dict = {}

    for _, row in df.iterrows():
        StateID = row['StateID']
        State= row['State']
        state_to_stateID_dict[State] = StateID
    
    conn.close()
    return state_to_stateID_dict

def create_stateID_to_state_dictionary(normalized_database_filename):

    conn = create_connection(normalized_database_filename)

    df = pd.read_sql_query ("SELECT * FROM State", conn)

    stateID_to_state_dict = {}

    for _, row in df.iterrows():
        StateID = row['StateID']
        State= row['State']
        stateID_to_state_dict[StateID] = State
    
    conn.close()
    return stateID_to_state_dict

state_to_stateID_dict = create_state_to_stateID_dictionary(normalized_database_filename)
stateID_to_state_dict = create_stateID_to_state_dictionary(normalized_database_filename)

In [ ]:
#Step 2 Create Municipality Table
def create_municipality_table(data_filename, normalized_database_filename):

  muni_list = []

  with open (data_filename) as file: 
    header = None

    for line in file:
      if not line.strip():
        continue
        
      if not header:
        header = line.strip().split('\t')
        continue

      data = line.strip().split('\t')
      muni = data[7].upper()
      state = data[9].upper()

      stateID= state_to_stateID_dict[state]
      
      muni_tuple = (muni, stateID)

      if muni_tuple not in muni_list:
        muni_list.append(muni_tuple)


  create_muni_table_sql = '''
    CREATE TABLE [Municipality] (
    [MunicipalityID] INTEGER NOT NULL PRIMARY KEY,
    [Municipality] TEXT NOT NULL,
    [StateID] INTEGER NOT NULL,
    FOREIGN KEY ([StateID]) REFERENCES State(StateID)
    );
    '''
  
  def insert_muni(conn, values):
    sql = 'INSERT into Municipality (Municipality, StateID) VALUES (?,?)'
    cur = conn.cursor()
    cur.executemany(sql,values)
    return (cur.lastrowid)
  
  conn = create_connection(normalized_database_filename)
  
  with conn:
    create_table(conn, create_muni_table_sql, "Municipality")
    insert_muni(conn, muni_list)
  conn.close()


create_municipality_table(data_filename, normalized_database_filename)

df = pd.read_sql_query("SELECT * FROM Municipality", conn)
display(df)

In [ ]:
# Create Munipality to MunipalityID Dictionary
def create_muni_to_muniID_dictionary(normalized_database_filename):

    conn = create_connection(normalized_database_filename)

    df = pd.read_sql_query ("SELECT * FROM Municipality", conn)

    muni_to_muniID_dict = {}

    for _, row in df.iterrows():
        MunicipalityID = row['MunicipalityID']
        Municipality = row['Municipality']
        StateID = row['StateID']
        State = stateID_to_state_dict[StateID]

        muni_state = Municipality + ', ' + State 
        muni_to_muniID_dict[muni_state] = MunicipalityID
    
    conn.close()
    return muni_to_muniID_dict

muni_to_muniID_dict = create_muni_to_muniID_dictionary(normalized_database_filename)

In [ ]:
#Step 3 Create Business Description Table
def create_business_description_table(data_filename, normalized_database_filename):
    description_list = []

    with open (data_filename) as file: 
        header = None

        for line in file:
            if not line.strip():
                continue
                
            if not header:
                header = line.strip().split('\t')
                continue

            data = line.strip().split('\t')
            category = data[1]
            category_decription = data[2]
                
            description_tuple = (category,category_decription)

            if description_tuple not in description_list:
                description_list.append(description_tuple)


        create_decription_table_sql = '''
            CREATE TABLE [Description] (
            [DescriptionID] INTEGER NOT NULL PRIMARY KEY,
            [Category] TEXT NOT NULL,
            [Description] TEXT NOT NULL);
            '''
    
    def insert_status(conn, values):
        sql = 'INSERT into Description (Category, Description) VALUES (?,?)'
        cur = conn.cursor()
        cur.executemany(sql,values)
        return (cur.lastrowid)
    
    conn = create_connection(normalized_database_filename)
    
    with conn:
        create_table(conn, create_decription_table_sql, "Description")
        insert_status(conn, description_list)
    conn.close()


create_business_description_table(data_filename, normalized_database_filename)

df = pd.read_sql_query ("SELECT * FROM Description", conn)
display (df)

In [ ]:
#Create Description To Description ID Dictionary
def create_description_to_descriptionID_dictionary(normalized_database_filename):

    conn = create_connection(normalized_database_filename)

    df = pd.read_sql_query ("SELECT * FROM Description", conn)

    description_to_descriptionID_dict = {}

    for _, row in df.iterrows():
        DescriptionID = row['DescriptionID']
        Category = row['Category']
        description_to_descriptionID_dict[Category] = DescriptionID
    
    conn.close()
    return description_to_descriptionID_dict

description_to_descriptionID_dict = create_description_to_descriptionID_dictionary(normalized_database_filename)

In [ ]:
#Step 4 Create Business Table
def create_business_table(data_filename, normalized_database_filename):
  
  business_list = []

  with open (data_filename) as file: 
    header = None

    for line in file:
      if not line.strip():
        continue
        
      if not header:
        header = line.strip().split('\t')
        continue

      data = line.strip().split('\t')
      business = data[5].upper()
      applicant = data[6].upper()
      Municipality = data[7].upper()
      State = data[9].upper()
      
      muni_state = (Municipality + ', ' + str(State))
      muni = muni_to_muniID_dict[muni_state]
      category = data[1] 

      if business == 'NO DATA':
        business = applicant
        
      business_tuple = (business, description_to_descriptionID_dict[category], muni)

      if business_tuple not in business_list:
        business_list.append(business_tuple)

    
  create_business_table_sql = '''
    
    CREATE TABLE [Business] (
    [BusinessID] INTEGER NOT NULL PRIMARY KEY,
    [Business] TEXT NOT NULL,
    [DescriptionID] INT NOT NULL,
    [MunicipalityID] INT NOT NULL,
    FOREIGN KEY ([DescriptionID]) REFERENCES Description(DescriptionID),
    FOREIGN KEY ([MunicipalityID]) REFERENCES Municipality(MunicipalityID)
    );
    '''
  
  def insert_business(conn, values):
    sql = 'INSERT into Business (Business, DescriptionID, MunicipalityID) VALUES (?,?,?)'
    cur = conn.cursor()
    cur.executemany(sql,values)
    return (cur.lastrowid)
  
  conn = create_connection(normalized_database_filename)
  
  with conn:
    create_table(conn, create_business_table_sql, "Business")
    insert_business(conn, business_list)
  conn.close()

create_business_table(data_filename, normalized_database_filename)

df = pd.read_sql_query("SELECT * FROM Business", conn)
display(df)

In [ ]:
#Create Business To BusinessID Dictionary
def create_business_to_businessID_dictionary(normalized_database_filename):

    conn = create_connection(normalized_database_filename)

    df = pd.read_sql_query ("SELECT * FROM Business", conn)

    business_to_businessID_dict = {}

    for _, row in df.iterrows():
        BusinessID = row['BusinessID']
        Business = row['Business']
        business_to_businessID_dict[Business] = BusinessID
    
    conn.close()
    return business_to_businessID_dict

business_to_businessID_dict = create_business_to_businessID_dictionary(normalized_database_filename)

In [ ]:
#Step 5 Create Status Table
def create_status_table(data_filename, normalized_database_filename):
    status_list = []

    with open (data_filename) as file: 
        header = None

        for line in file:
            if not line.strip():
                continue
                
            if not header:
                header = line.strip().split('\t')
                continue

            data = line.strip().split('\t')
            status = data[3]
                
            status_tuple = (status,)



            if status_tuple not in status_list:
                status_list.append(status_tuple)


        create_status_table_sql = '''
            CREATE TABLE [Status] (
            [StatusID] INTEGER NOT NULL PRIMARY KEY,
            [Status] TEXT NOT NULL);
            '''
    
    def insert_status(conn, values):
        sql = 'INSERT into Status (Status) VALUES (?)'
        cur = conn.cursor()
        cur.executemany(sql,values)
        return (cur.lastrowid)
    
    conn = create_connection(normalized_database_filename)
    
    with conn:
        create_table(conn, create_status_table_sql, "Status")
        insert_status(conn, status_list)
    conn.close()

create_status_table(data_filename, normalized_database_filename)

df = pd.read_sql_query("SELECT * FROM Status", conn)
display(df)

In [ ]:

def create_status_to_statusID_dictionary(normalized_database_filename):

    conn = create_connection(normalized_database_filename)

    df = pd.read_sql_query ("SELECT * FROM Status", conn)

    status_to_statusID_dict = {}

    for _, row in df.iterrows():
        StatusID = row['StatusID']
        Status = row['Status']
        status_to_statusID_dict[Status] = StatusID
    
    conn.close()
    return status_to_statusID_dict

status_to_statusID_dict = create_status_to_statusID_dictionary(normalized_database_filename)


In [ ]:
#Step 6 Create Applicant Table
def create_applicant_table(data_filename, normalized_database_filename):

  applicant_list = []

  with open (data_filename) as file: 
    header = None

    for line in file:
      if not line.strip():
        continue
        
      if not header:
        header = line.strip().split('\t')
        continue

      data = line.strip().split('\t')

      applicant = data[6].upper()
      phoneno = data[8]
        
      applicant_tuple = (applicant, phoneno)

      if applicant_tuple not in applicant_list:
        applicant_list.append(applicant_tuple)

  create_applicant_table_sql = '''
    CREATE TABLE [Applicants] (
    [ApplicantID] INTEGER NOT NULL PRIMARY KEY,
    [Applicant] TEXT NOT NULL,
    [PhoneNo] TEXT
    );
    '''
  
  def insert_licence(conn, values):
    sql = 'INSERT into Applicants (Applicant,PhoneNo) VALUES (?,?)'
    cur = conn.cursor()
    cur.executemany(sql,values)
    return (cur.lastrowid)
  
  conn = create_connection(normalized_database_filename)
  
  with conn:
    create_table(conn, create_applicant_table_sql, "Applicants")
    insert_licence(conn, applicant_list)
  conn.close()

create_applicant_table(data_filename, normalized_database_filename)

df = pd.read_sql_query("SELECT * FROM Applicants", conn)
display(df)

In [ ]:
#Create Applicant To ApplicantID Dictionary
def create_applicant_to_applicantID_dictionary(normalized_database_filename):

  conn = create_connection(normalized_database_filename)

  df = pd.read_sql_query ("SELECT * FROM Applicants", conn)

  applicant_to_applicantID_dict = {}

  for _, row in df.iterrows():
      ApplicantID = row['ApplicantID']
      Applicant = row['Applicant']
      phoneNo = row['PhoneNo']
      applicant_to_applicantID_dict[Applicant+phoneNo] = ApplicantID
  
  conn.close()
  return applicant_to_applicantID_dict

applicant_to_applicantID_dict = create_applicant_to_applicantID_dictionary(normalized_database_filename)

In [50]:
#Step 7 Create License Table
def create_license_table(data_filename, normalized_database_filename):

  license_list = []
  from datetime import datetime
  with open (data_filename) as file: 
    header = None

    for line in file:
      if not line.strip():
        continue
        
      if not header:
        header = line.strip().split('\t')
        continue

      data = line.strip().split('\t')
      licenseNo = data[0]
      exp_date = data[4].strip()
      exp_date = datetime.strptime(exp_date, '%m/%d/%Y')
      exp_date = exp_date.strftime('%Y-%m-%d')
      applicant = data[6].upper()
      phoneno = data[8]

      applicantkey = applicant + phoneno
      business = data[5].upper()
      if business == 'NO DATA':
        business = applicant
      
      businessID = business_to_businessID_dict[business]
      applicantID = applicant_to_applicantID_dict[applicantkey]
      license_tuple = (licenseNo, exp_date,businessID, applicantID)

      if license_tuple not in license_list:
        license_list.append(license_tuple)

  create_license_table_sql = '''
    CREATE TABLE [Licenses] (
    [LicenseID] INTEGER NOT NULL PRIMARY KEY,
    [LicenseNo] TEXT NOT NULL,
    [LicensesExpDate] TEXT NOT NULL,
    [BusinessID] INTEGER NOT NULL,
    [ApplicantID] INTEGER NOT NULL,
    FOREIGN KEY ([businessID]) REFERENCES Business(businessID),
    FOREIGN KEY ([ApplicantID]) REFERENCES Applicants(ApplicantID)
    );
    '''
  
  def insert_licence(conn, values):
    sql = 'INSERT into Licenses (LicenseNo,LicensesExpDate,BusinessID,ApplicantID) VALUES (?,?,?,?)'
    cur = conn.cursor()
    cur.executemany(sql,values)
    return (cur.lastrowid)
  
  conn = create_connection(normalized_database_filename)
  
  with conn:
    create_table(conn, create_license_table_sql, "Licenses")
    insert_licence(conn, license_list)
  conn.close()

create_license_table(data_filename, normalized_database_filename)

df = pd.read_sql_query("SELECT * FROM Licenses", conn)
display(df)

,LicenseID,LicenseNo,LicensesExpDate,BusinessID,ApplicantID
0,1,SEC11-515268,2025-06-30,1,1
1,2,CAG15-10033688,2025-09-30,2,2
2,3,CAG16-10034349,2025-09-30,3,3
3,4,PLU14-10027646,2025-12-31,4,4
4,5,SEC23-10062768,2025-05-31,5,5
...,...,...,...,...,...
5055,5056,PLJ12-10023741,2026-12-31,4875,4722
5056,5057,SE123-10063546,2025-07-31,4876,4723
5057,5058,SE123-10063619,2025-08-31,4877,4724
5058,5059,HCQ20-10057101,2025-12-31,4878,4725


In [ ]:
#Create License To LicenseID Dictionary

def create_license_to_licenseID_dictionary(normalized_database_filename):

  conn = create_connection(normalized_database_filename)

  df = pd.read_sql_query ("SELECT * FROM Licenses", conn)

  license_to_licenseID_dict = {}

  for _, row in df.iterrows():
      LicenseID = row['LicenseID']
      License = row['LicenseNo']
      license_to_licenseID_dict[License] = LicenseID
  
  conn.close()
  return license_to_licenseID_dict

license_to_licenseID_dict = create_license_to_licenseID_dictionary(normalized_database_filename)

In [55]:
#Query 1: Which type of license is most frequently issued?
sql_statement_query_1 = '''
SELECT
Description,
COUNT(*) AS LicenseCount
FROM Licenses

JOIN Business ON Licenses.BusinessID = Business.BusinessID
JOIN Description ON Business.DescriptionID = Description.DescriptionID
GROUP BY Description.Description
ORDER BY LicenseCount DESC;
'''
df = pd.read_sql_query(sql_statement_query_1, conn)
df

,Description,LicenseCount
0,LIGHT COMMERCIAL CONTRACTOR,834
1,SE 1ST CLASS,616
2,COLLECTION AGENCY,613
3,PLUMBERS JOURNEYMEN,437
4,SE CHIEF,416
5,SE 2ND CLASS,223
6,PLUMBERS MASTER,188
7,HEATING CONTRACTOR,184
8,MASTER ELECTRICIAN,169
9,SUBCONTRACTOR,146


In [46]:
#Query 2: Which applicants have license applications that are missing contact information?
sql_statement_query_2 = '''
SELECT DISTINCT Licenses.LicenseNo, Business.Business, Applicants.Applicant, Applicants.PhoneNo
FROM Licenses

INNER JOIN Business ON Licenses.BusinessID = Business.BusinessID
INNER JOIN Applicants ON Licenses.ApplicantID = Applicants.ApplicantID

WHERE Applicants.PhoneNo = 'NO DATA'
ORDER BY Business.Business, Applicants.Applicant;
'''
df = pd.read_sql_query(sql_statement_query_2, conn)
df

,LicenseNo,Business,Applicant,PhoneNo
0,LTC24-10066924,3 BROTHERS PAINT CO LLC,ERIC WALTON,NO DATA
1,TTR24-10066522,5T TOWING AND RECOVERY LLC,OMAR TIRADO,NO DATA
2,LTC22-10061715,6 STARS PROPERTY MANAGEMENT INC.,QUENTIN YANCEY,NO DATA
3,LTC25-10068620,716 BROTHERS CONSTRUCTION INC,LUIS MAYANCELA,NO DATA
4,LTC23-10062151,716 CLEANUP & RESTORATION CORP.,AARON REGDOS,NO DATA
...,...,...,...,...
886,JRP24-10066240,ZACHERY KIBLIN,ZACHERY KIBLIN,NO DATA
887,HMC23-10065238,ZBUILT LLC,ZACHARY LEWIS,NO DATA
888,CAG25-10069693,ZENDABLE LLC,MELISSA BONNEVILLE,NO DATA
889,CAG11-553953,ZENTA RECOVERIES INC.,ZENTA RECOVERIES INC.,NO DATA


In [ ]:
#Query 3: Identify applicants whose licenses are due to expire within the next month (May 1st -May 31st).
sql_statement_query_3 = '''
SELECT
Licenses.LicenseNo,
Licenses.LicensesExpDate,
Applicants.Applicant,
Applicants.PhoneNo
FROM Licenses

INNER JOIN Applicants ON Licenses.ApplicantID = Applicants.ApplicantID

WHERE Licenses.LicensesExpDate BETWEEN '2025-05-01' AND '2025-05-31' AND Applicants.PhoneNo != 'NO DATA'
ORDER BY Licenses.LicensesExpDate ASC;
'''

df = pd.read_sql_query(sql_statement_query_3, conn)
df

,LicenseNo,LicensesExpDate,Applicant,PhoneNo
0,STP14-10028086,2025-05-03,RICHARD E DANIELS,(716)835-0345
1,STP19-10050824,2025-05-28,PHILIP ROMA,(716)668-7885
2,SEC23-10062768,2025-05-31,VINCENT ZIOLKOWSKI,(716)425-4891
3,SE223-10062727,2025-05-31,LEON SMITH,(716)818-7730
4,SE211-160847,2025-05-31,JEFFREY CANDINO,(716)564-0184
...,...,...,...,...
92,SE117-10040371,2025-05-31,LOUIS PRZYSTAL,(716)823-0067
93,SE223-10062758,2025-05-31,CHRISTOPHER CHAPMAN,(716)308-0420
94,SE221-10059137,2025-05-31,RYAN WINTER,(716)912-7437
95,SE119-10050509,2025-05-31,JEFFREY LAVOCAT,(716)352-4423


In [56]:
#Query 4: Which businesses, that hold a Demolition Level 3 license and are based in Buffalo, New York?
sql_statement_query_4 = '''
SELECT 
Business.Business,
Municipality.Municipality,
Description.Description,
State.State
FROM Business

JOIN Municipality ON Business.MunicipalityID = Municipality.MunicipalityID
JOIN Description ON Business.DescriptionID = Description.DescriptionID
JOIN State ON Municipality.StateID = State.StateID

WHERE UPPER(Municipality.Municipality) = 'BUFFALO' AND Description.DescriptionID = 10
ORDER BY Business.Business;
'''
df = pd.read_sql_query(sql_statement_query_4, conn)
df

,Business,Municipality,Description,State
0,"ACP BUFFALO, LLC",BUFFALO,DEMOLITION GRADE 3,NY
1,ATLANTIC GARAGES,BUFFALO,DEMOLITION GRADE 3,NY
2,DEVCON US LLC,BUFFALO,DEMOLITION GRADE 3,NY
3,ELLICOTT CONSTRUCTION CO INC,BUFFALO,DEMOLITION GRADE 3,NY
4,FRONTIER INDUSTRIAL CORP,BUFFALO,DEMOLITION GRADE 3,NY
5,GEITER DONE OF WNY INC.,BUFFALO,DEMOLITION GRADE 3,NY
6,HANNAH DEMOLITION INC,BUFFALO,DEMOLITION GRADE 3,NY
7,LEE'S ENVIRONMENTAL SERVICES INC,BUFFALO,DEMOLITION GRADE 3,NY
8,ONTARIO SPECIALTY CONTRACTING,BUFFALO,DEMOLITION GRADE 3,NY
9,TOTAL WRECKING & ENVIRONMENTAL LLC,BUFFALO,DEMOLITION GRADE 3,NY


In [57]:
#Query 5: Identify businesses that hold licenses for both plumbing and heating?
sql_statement_query_4 = '''
SELECT 
Business.Business,
GROUP_CONCAT(DISTINCT DescriptionID) AS DescriptionIDs
FROM Business

WHERE DescriptionID IN (10, 14, 26)
GROUP BY Business.Business
HAVING COUNT(DISTINCT DescriptionID) > 2;
'''
df = pd.read_sql_query(sql_statement_query_4, conn)
df

,Business,DescriptionIDs
0,"ACP BUFFALO, LLC","10,26,14"
1,EMPIRE BUILDING DIAGNOSTICS INC,"14,10,26"
